In [ ]:
import requests, zipfile
from io import BytesIO
import xml.etree.ElementTree as ET
import json
import time
import difflib
import openai
from openai import OpenAI

# 🔑 API 키
DART_API_KEY = ""
OPENAI_API_KEY = ""  # 너의 OpenAI 키 입력
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ 설정
DEFAULT_YEAR = "2023"
DEFAULT_REPRT_CODE = "11011"


In [ ]:
# ✅ 기업 리스트 불러오기
def load_corp_list(api_key):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)
    z = zipfile.ZipFile(BytesIO(res.content))
    xml_content = z.read(z.namelist()[0])
    root = ET.fromstring(xml_content)
    return [{"corp_code": c.findtext("corp_code"), "corp_name": c.findtext("corp_name")} for c in root.findall("list")]

# ✅ 기업 개요
def collect_profile(corp_code):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": DART_API_KEY, "corp_code": corp_code}
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return {}
    return {
        "회사명": res.get("corp_name"),
        "대표자": res.get("ceo_nm"),
        "사업자등록번호": res.get("bizr_no"),
        "설립일": res.get("est_dt"),
        "업종코드": res.get("industry_code"),
        "주소": res.get("adres"),
        "지주회사": res.get("hm_ownr_corp_nm"),
        "상장여부": "상장" if res.get("stock_code") else "비상장"
    }

# ✅ 재무정보
def collect_finance(corp_code):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE,
        "fs_div": "CFS"
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item["account_nm"] == "매출액":
            return item["thstrm_amount"]
    return None

# ✅ 고용정보
def collect_employees(corp_code):
    url = "https://opendart.fss.or.kr/api/emplSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item.get("se") == "합계":
            return item.get("thstrm") or item.get("thstrm_nm")
    return None

# ✅ 계열사
def collect_affiliates(corp_code):
    url = "https://opendart.fss.or.kr/api/dsclSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return []
    return [item.get("corp_name") for item in res.get("list", []) if item.get("corp_name")]

# ✅ GPT 요약
def gpt_summary(profile, sales, emp, affiliates):
    categories = "패션, 물류, 핀테크, 유통, 콘텐츠, 플랫폼, 커머스, IT, 미디어, 제조, 기타"
    keywords = "예시: #2030 #남성패션 #MZ, 주요 사업 아이템이나 고객에 대한 정보 제공"
    investor_types = "#공격형, #안정형, #혼합형"

    prompt = f"""당사는 다음과 같은 기업입니다:

[기업 개요]
- 회사명: {profile.get("회사명")}
- 대표자: {profile.get("대표자")}
- 설립일: {profile.get("설립일")}
- 업종코드: {profile.get("업종코드")}
- 주소: {profile.get("주소")}
- 상장여부: {profile.get("상장여부")}

[재무 및 인력]
- 총매출: {sales or '매출 정보 없음'}
- 총인원 수: {emp or '인력 정보 없음'}

[계열사]
- {', '.join(affiliates) if affiliates else '계열사 정보 없음'}

넌 20년 된 투자 전문가야. 일반인에게 설명할 거야.

1. 이 기업이 속한 주요 산업 분야를 반드시 다음 중 하나로 판단해서 출력해줘:
{categories}

2. 다음 중 최대 2개의 키워드를 골라 해시태그로 출력해줘:
{keywords}

3. 반드시 다음 중 하나를 세 번째 키워드로 붙여줘 (항상 마지막에 위치):
{investor_types}

4. 이 기업을 전문가 관점에서 핵심 요점만 뽑아 한 문장으로 요약해줘.

출력 형식:

{{
  "주요 분야": "패션",
  "키워드": [" #2030 #남성패션 #MZ, 주요 사업 아이템이나 고객에 대한 정보 제공 #공격형"],
  "한 문장 요약": "이 기업은 ..."
}}"""
    try:
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        # GPT 응답 확인 (디버깅용)
        # print("GPT 응답:", res.choices[0].message.content)
        return json.loads(res.choices[0].message.content)
    except Exception as e:
        return {
            "주요 분야": "정보 없음",
            "키워드": ["#정보없음"],
            "한 문장 요약": f"❌ GPT 요약 실패: {e}"
        }

# ✅ 리액트용 카드로 변환
def convert_result_to_react_json(result_dict, image_url=None):
    return {
        "name": result_dict["회사명"],
        "src": image_url or "",
        "itemList": [
            {
                "type": "text",
                "data": [result_dict["GPT 요약"]]
            },
            {
                "type": "text",
                "data": [result_dict["상장여부"]]
            },
            {
                "type": "text",
                "data": [f"{int(result_dict['총매출(천원)'])/1e8:.0f}억" if result_dict['총매출(천원)'] not in ["정보 없음", None] else "정보 없음"]
            },
            {
                "type": "text",
                "data": [f"{result_dict['총인원 수']}명" if result_dict['총인원 수'] not in ["정보 없음", None] else "정보 없음"]
            },
            {
                "type": "tag",
                "data": [result_dict["주요 분야"]] + result_dict.get("키워드", [])[:-1]
            },
            {
                "type": "tag",
                "data": result_dict.get("키워드", [])
            }
        ]
    }

# ✅ 실행: 사용자 입력 → 자동 처리
keyword = input("🔍 검색할 기업명을 입력하세요: ").strip()
corp_list = load_corp_list(DART_API_KEY)

# ✅ 유사 기업 정렬 (정확도 순)
corp_names = [c["corp_name"] for c in corp_list]
closest = difflib.get_close_matches(keyword, corp_names, n=20, cutoff=0.3)
matches = [c for name in closest for c in corp_list if c["corp_name"] == name]

if not matches:
    print("❌ 관련된 기업을 찾을 수 없습니다.")
else:
    print(f"\n✅ '{keyword}'와 유사한 기업 {len(matches)}개 자동 출력 중...\n")

    for idx, match in enumerate(matches, 1):
        print(f"📦 [{idx}] {match['corp_name']}")

        try:
            corp_code = match["corp_code"]
            profile = collect_profile(corp_code)
            sales = collect_finance(corp_code)
            emp = collect_employees(corp_code)
            affiliates = collect_affiliates(corp_code)
            summary = gpt_summary(profile, sales, emp, affiliates)

            result_dict = {
                **profile,
                "총매출(천원)": sales or "정보 없음",
                "총인원 수": emp or "정보 없음",
                "계열사": affiliates,
                "주요 분야": summary.get("주요 분야"),
                "키워드": summary.get("키워드"),
                "GPT 요약": summary.get("한 문장 요약")
            }

            react_card = convert_result_to_react_json(result_dict)
            print(json.dumps(react_card, ensure_ascii=False, indent=2))
            print("\n" + "-" * 80 + "\n")

        except Exception as e:
            print(f"❌ [{match['corp_name']}] 처리 중 오류 발생: {e}")
            print("\n" + "-" * 80 + "\n")

🔍 검색할 기업명을 입력하세요: 두산

✅ '두산'와 유사한 기업 24개 자동 출력 중...

📦 [1] 두산
{
  "name": "(주)두산",
  "src": "",
  "itemList": [
    {
      "type": "text",
      "data": [
        "이 기업은 1933년에 설립된 역사 있는 제조 기업으로 다양한 사업 분야에서 성장하고 있는 공격형 기업이다."
      ]
    },
    {
      "type": "text",
      "data": [
        "상장"
      ]
    },
    {
      "type": "text",
      "data": [
        "191301억"
      ]
    },
    {
      "type": "text",
      "data": [
        "정보 없음"
      ]
    },
    {
      "type": "tag",
      "data": [
        "제조"
      ]
    },
    {
      "type": "tag",
      "data": [
        "#역사있는기업 #다양한사업분야 #공격형"
      ]
    }
  ]
}

--------------------------------------------------------------------------------

📦 [2] 선두산업
{
  "name": "주식회사 선두산업",
  "src": "",
  "itemList": [
    {
      "type": "text",
      "data": [
        "이 기업은 2019년에 설립된 산업 분야가 기타인 비상장 기업으로, 물류에 초점을 맞춘 공격적인 전략을 취하고 있다."
      ]
    },
    {
      "type": "text",
      "data": [
        "비상장"
      ]
    },
    {
      